In [0]:
This is a simple

In [0]:
# File location and type
file_location = "/FileStore/tables/feature_matrix-2.csv"
file_type = "csv"

# CSV options
data=spark.read.csv(file_location,header=True,inferSchema=True)

In [0]:
data.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Speed: double (nullable = true)
 |-- Distance: double (nullable = true)
 |-- Distance_x: double (nullable = true)
 |-- Distance_y: double (nullable = true)
 |-- PCI: integer (nullable = true)
 |-- PCI_64: integer (nullable = true)
 |-- PCI_65: integer (nullable = true)
 |-- PCI_302: integer (nullable = true)



In [0]:
data.show(10)

+---+---------+-----------------+-----+------------------+--------------------+--------------------+---+------+------+-------+
|_c0|Longitude|         Latitude|Speed|          Distance|          Distance_x|          Distance_y|PCI|PCI_64|PCI_65|PCI_302|
+---+---------+-----------------+-----+------------------+--------------------+--------------------+---+------+------+-------+
|  0|12.520112|        55.780073|21.51|0.5530644296850808|-0.00459719999999...|-0.00336900000000...| 64|     1|     0|      0|
|  1|12.520119|        55.780071|20.93| 0.553103502794036|-0.00459920000000...|-0.00336200000000...| 64|     1|     0|      0|
|  2|12.520127|         55.78007|20.36|0.5530163642392476|-0.00460019999999...|-0.00335399999999...| 64|     1|     0|      0|
|  3|12.520135|55.78006800000001|19.78| 0.553032643909539|-0.00460219999999...|-0.00334600000000...| 64|     1|     0|      0|
|  4|12.520142|        55.780067|19.21|0.5529700878366755|-0.00460319999999...|-0.00333900000000...| 64|     1|

In [0]:
d=data.select('Latitude')
d.show(10)

+-----------------+
|         Latitude|
+-----------------+
|        55.780073|
|        55.780071|
|         55.78007|
|55.78006800000001|
|        55.780067|
|        55.780065|
|        55.780064|
|        55.780062|
|        55.780061|
|        55.780059|
+-----------------+
only showing top 10 rows



In [0]:
data.columns

Out[8]: ['_c0',
 'Longitude',
 'Latitude',
 'Speed',
 'Distance',
 'Distance_x',
 'Distance_y',
 'PCI',
 'PCI_64',
 'PCI_65',
 'PCI_302']

In [0]:
from pyspark.ml.feature import VectorAssembler
vect=VectorAssembler(inputCols=['Longitude','Latitude','Distance','Distance_x','Distance_y','PCI','PCI_64','PCI_65','PCI_302'],outputCol="independent")
output=vect.transform(data)

In [0]:
output.select('independent').show(5)

+--------------------+
|         independent|
+--------------------+
|[12.520112,55.780...|
|[12.520119,55.780...|
|[12.520127,55.780...|
|[12.520135,55.780...|
|[12.520142,55.780...|
+--------------------+
only showing top 5 rows



In [0]:
output.show(5)

+---+---------+-----------------+-----+------------------+--------------------+--------------------+---+------+------+-------+--------------------+
|_c0|Longitude|         Latitude|Speed|          Distance|          Distance_x|          Distance_y|PCI|PCI_64|PCI_65|PCI_302|         independent|
+---+---------+-----------------+-----+------------------+--------------------+--------------------+---+------+------+-------+--------------------+
|  0|12.520112|        55.780073|21.51|0.5530644296850808|-0.00459719999999...|-0.00336900000000...| 64|     1|     0|      0|[12.520112,55.780...|
|  1|12.520119|        55.780071|20.93| 0.553103502794036|-0.00459920000000...|-0.00336200000000...| 64|     1|     0|      0|[12.520119,55.780...|
|  2|12.520127|         55.78007|20.36|0.5530163642392476|-0.00460019999999...|-0.00335399999999...| 64|     1|     0|      0|[12.520127,55.780...|
|  3|12.520135|55.78006800000001|19.78| 0.553032643909539|-0.00460219999999...|-0.00334600000000...| 64|     1| 

In [0]:
final_data=output.select('independent','Speed')
final_data.show(5)

+--------------------+-----+
|         independent|Speed|
+--------------------+-----+
|[12.520112,55.780...|21.51|
|[12.520119,55.780...|20.93|
|[12.520127,55.780...|20.36|
|[12.520135,55.780...|19.78|
|[12.520142,55.780...|19.21|
+--------------------+-----+
only showing top 5 rows



In [0]:
from pyspark.ml.regression import LinearRegression
train_data,test_data=final_data.randomSplit([0.75,0.25])
regressor=LinearRegression(featuresCol='independent',labelCol='Speed')
regressor=regressor.fit(train_data)

In [0]:
regressor.intercept

Out[23]: -435.82239131905425

In [0]:
regressor.coefficients

Out[24]: DenseVector([160.4475, -27.8816, 5.7044, -27.8823, 160.4473, 0.0001, 0.0715, -0.0677, 0.0176])

In [0]:
pred=regressor.evaluate(test_data)

In [0]:
pred.predictions.show(10)

+--------------------+-----+------------------+
|         independent|Speed|        prediction|
+--------------------+-----+------------------+
|[12.511929,55.783...|33.74|18.641517510189544|
|[12.51197,55.7836...|33.29|18.632314624989874|
|[12.512045,55.783...|23.34| 18.60269853089011|
|[12.51209,55.7836...|17.19| 18.62433993503538|
|[12.512103,55.783...|28.34|18.593121503125815|
|[12.512109,55.783...|28.76| 18.59211792146482|
|[12.512116,55.783...|29.18|18.591084138054498|
|[12.512123,55.783...| 29.6| 18.59005330830354|
|[12.512135,55.783...|15.88|18.625416383568677|
|[12.512141,55.783...|15.67|18.625541891543094|
+--------------------+-----+------------------+
only showing top 10 rows



In [0]:
print("The mean absolute error is {}".format(pred.meanAbsoluteError))

The mean absolute error is 9.477572080483217
